In [67]:
import warnings
import time
import os
import geopandas as gpd
RASTERIO_BEST_PRACTICES = dict(  # See https://github.com/pangeo-data/cog-best-practices
    CURL_CA_BUNDLE="/etc/ssl/certs/ca-certificates.crt",
    GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR",
    AWS_NO_SIGN_REQUEST="YES",
    GDAL_MAX_RAW_BLOCK_CACHE_SIZE="200000000",
    GDAL_SWATH_SIZE="200000000",
    VSI_CURL_CACHE_SIZE="200000000",
)
os.environ.update(RASTERIO_BEST_PRACTICES)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import rasterio
import rasterio.warp
import rasterio.mask
import shapely.geometry
import geopandas
import dask_geopandas
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy.stats import spearmanr
from scipy.linalg import LinAlgWarning
from dask.distributed import Client
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import mean_squared_error

warnings.filterwarnings(action="ignore", category=LinAlgWarning, module="sklearn")

import pystac_client
import planetary_computer as pc

# Function define

In [68]:
def featurize(input_img, model, device):
    """Helper method for running an image patch through the model.

    Args:
        input_img (np.ndarray): Image in (C x H x W) format with a dtype of uint8.
        model (torch.nn.Module): Feature extractor network
    """
    assert len(input_img.shape) == 1
    input_img = torch.from_numpy(input_img / 255.0).float()
    input_img = input_img.to(device)
    with torch.no_grad():
        feats = model(input_img.unsqueeze(0)).cpu().numpy()
    return feats



# RCF RGB

In [69]:
class RCF(nn.Module):
    """A model for extracting Random Convolution Features (RCF) from input imagery."""

    def __init__(self, num_features=16, kernel_size=3, num_input_channels):
        super(RCF, self).__init__()

        # We create `num_features / 2` filters so require `num_features` to be divisible by 2
        assert num_features % 2 == 0

        self.conv1 = nn.Conv2d(
            num_input_channels,
            num_features // 2,
            kernel_size=kernel_size,
            stride=1,
            padding=0,
            dilation=1,
            bias=True,
        )

        nn.init.normal_(self.conv1.weight, mean=0.0, std=1.0)
        nn.init.constant_(self.conv1.bias, -1.0)

    def forward(self, x):
        x1a = F.relu(self.conv1(x), inplace=True)
        x1b = F.relu(-self.conv1(x), inplace=True)

        x1a = F.adaptive_avg_pool2d(x1a, (1, 1)).squeeze()
        x1b = F.adaptive_avg_pool2d(x1b, (1, 1)).squeeze()

        if len(x1a.shape) == 1:  # case where we passed a single input
            return torch.cat((x1a, x1b), dim=0)
        elif len(x1a.shape) == 2:  # case where we passed a batch of > 1 inputs
            return torch.cat((x1a, x1b), dim=1)
device = torch.device("cuda")


# Query

In [120]:
def query_7(points):
    """
    Find a STAC item for points in the `points` DataFrame

    Parameters
    ----------
    points : geopandas.GeoDataFrame
        A GeoDataFrame

    Returns
    -------
    geopandas.GeoDataFrame
        A new geopandas.GeoDataFrame with a `stac_item` column containing the STAC
        item that covers each point.
    """
    intersects = shapely.geometry.mapping(points.unary_union.convex_hull)

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    
    search = catalog.search(
    collections=["landsat-c2-l2"], #  collections=["landsat-8-c2-l2"],
    intersects=intersects,
    datetime=["2012-01-01", "2013-12-31"],
    query={"eo:cloud_cover": {"lt": 30}},
    limit=500,
    
)
    
    

    # The time frame in which we search for non-cloudy imagery
    
    ic = search.get_all_items_as_dict()
    print(f"Returned {len(ic)} Items")
    
    features = ic["features"]
    features_d = {item["id"]: item for item in features}

    data = {
        "eo:cloud_cover": [],
        "geometry": [],
    }

    index = []

    for item in features:
        data["eo:cloud_cover"].append(item["properties"]["eo:cloud_cover"])
        data["geometry"].append(shapely.geometry.shape(item["geometry"]))
        index.append(item["id"])

    items = geopandas.GeoDataFrame(data, index=index, geometry="geometry").sort_values(
        "eo:cloud_cover"
    )
    point_list = points.geometry.tolist()

    point_items = []
    for point in point_list:
        covered_by = items[items.covers(point)]
        if len(covered_by):
            point_items.append(features_d[covered_by.index[0]])
        else:
            # There weren't any scenes matching our conditions for this point (too cloudy)
            point_items.append(None)

    return points.assign(stac_item=point_items)

In [141]:
def query_8(points):
    """
    Find a STAC item for points in the `points` DataFrame

    Parameters
    ----------
    points : geopandas.GeoDataFrame
        A GeoDataFrame

    Returns
    -------
    geopandas.GeoDataFrame
        A new geopandas.GeoDataFrame with a `stac_item` column containing the STAC
        item that covers each point.
    """
    intersects = shapely.geometry.mapping(points.unary_union.convex_hull)

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    
    search = catalog.search(
    collections=["landsat-8-c2-l2"],
    intersects=intersects,
    datetime=["2013-01-01", "2014-12-31"],
    query={"eo:cloud_cover": {"lt": 10}},
    limit=500,
    
)
    
    

    # The time frame in which we search for non-cloudy imagery
    
    ic = search.get_all_items_as_dict()
    print(f"Returned {len(ic)} Items")
    
    features = ic["features"]
    features_d = {item["id"]: item for item in features}

    data = {
        "eo:cloud_cover": [],
        "geometry": [],
    }

    index = []

    for item in features:
        data["eo:cloud_cover"].append(item["properties"]["eo:cloud_cover"])
        data["geometry"].append(shapely.geometry.shape(item["geometry"]))
        index.append(item["id"])

    items = geopandas.GeoDataFrame(data, index=index, geometry="geometry").sort_values(
        "eo:cloud_cover"
    )
    point_list = points.geometry.tolist()

    point_items = []
    for point in point_list:
        covered_by = items[items.covers(point)]
        if len(covered_by):
            point_items.append(features_d[covered_by.index[0]])
        else:
            # There weren't any scenes matching our conditions for this point (too cloudy)
            point_items.append(None)

    return points.assign(stac_item=point_items)

In [142]:
class CustomDataset(Dataset):
    def __init__(self, points, fns, buffer=500):
        self.points = points
        self.fns = fns
        self.buffer = buffer

    def __len__(self):
        return self.points.shape[0]

    def __getitem__(self, idx):

        lon, lat = self.points[idx]
        fn = self.fns[idx]

        if fn is None:
            return None
        else:
            point_geom = shapely.geometry.mapping(shapely.geometry.Point(lon, lat))

            with rasterio.Env():
                with rasterio.open(fn, "r") as f:
                    point_geom = rasterio.warp.transform_geom(
                        "epsg:4326", f.crs.to_string(), point_geom
                    )
                    point_shape = shapely.geometry.shape(point_geom)
                    mask_shape = point_shape.buffer(self.buffer).envelope
                    mask_geom = shapely.geometry.mapping(mask_shape)
                    try:
                        out_image, out_transform = rasterio.mask.mask(
                            f, [mask_geom], crop=True
                        )
                    except ValueError as e:
                        if "Input shapes do not overlap raster." in str(e):
                            return None

            out_image = out_image / 255.0
            out_image = torch.from_numpy(out_image).float()
            return out_image

def extract_features(train_dataset, model, num_features):
    dataloader = DataLoader(
        train_dataset,
        batch_size=4,
        shuffle=False,
        num_workers=os.cpu_count() * 2,
        collate_fn=lambda x: x,
        pin_memory=False,
    )

    x_train = np.zeros((train_dataset.points.shape[0], num_features), dtype=float)

    tic = time.time()
    i = 0

    for images in dataloader:
        for image in images:
            if image is not None:
                # A full image should be ~101x101 pixels (i.e. ~1km^2 at a 10m/px spatial
                # resolution), however we can receive smaller images if an input point
                # happens to be at the edge of a S2 scene (a literal edge case). To deal
                # with these (edge) cases we crudely drop all images where the spatial
                # dimensions aren't both greater than 20 pixels.
                if image.shape[1] >= 20 and image.shape[2] >= 20:
                    image = image.to(device)
                    with torch.no_grad():
                        feats = model(image.unsqueeze(0)).cpu().numpy()

                    x_train[i] = feats
                else:
                    # this happens if the point is close to the edge of a scene
                    # (one or both of the spatial dimensions of the image are very small)
                    pass
            else:
                pass  # this happens if we do not find a S2 scene for some point

            if i % 1000 == 0:
                print(
                    f"{i}/{train_dataset.points.shape[0]} -- {i / train_dataset.points.shape[0] * 100:0.2f}%"
                    + f" -- {time.time()-tic:0.2f} seconds"
                )
                tic = time.time()
            i += 1

    return x_train


# Read dataset

In [143]:
df = pd.read_csv(

    "https://drive.google.com/uc?export=download&id=1vaZxJap_x1iyf-ytkDo13Dy_7ETdWsO3",  # noqa: E501
    index_col=0,
    na_values=[0,-999]
).dropna()
points = df[["lon", "lat"]]
houseprice = df["houseprice"]

gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.lon, df.lat))
gdf


,ID,lon,lat,houseprice,City,geometry
Unnamed: 0,,,,,,
243,"153,63",35.253528,0.461746,0.064764,Eldoret,POINT (35.25353 0.46175)
244,"153,77",35.266255,0.461746,0.086711,Eldoret,POINT (35.26626 0.46175)
254,"152,58",35.248983,0.462655,0.167129,Eldoret,POINT (35.24898 0.46266)
255,"152,72",35.261710,0.462655,1.091892,Eldoret,POINT (35.26171 0.46266)
265,"151,53",35.244437,0.463564,0.745180,Eldoret,POINT (35.24444 0.46356)
...,...,...,...,...,...,...
1573,"27,66",37.067096,-1.025716,7.017600,Thika,POINT (37.06710 -1.02572)
1574,"27,78",37.078005,-1.025716,7.017600,Thika,POINT (37.07800 -1.02572)
1590,"26,72",37.072550,-1.024807,5.070685,Thika,POINT (37.07255 -1.02481)


In [144]:
gdf.groupby('City').count()

,ID,lon,lat,houseprice,geometry
City,,,,,
Eldoret,590,590,590,590,590
Embu,329,329,329,329,329
Garissa,275,275,275,275,275
Kakamega,411,411,411,411,411
Kericho,244,244,244,244,244
Kisumu,167,167,167,167,167
Kitui,106,106,106,106,106
Machakos,264,264,264,264,264
Malindi,204,204,204,204,204


# Load data

In [148]:
light = pd.read_csv(
    "https://drive.google.com/uc?export=download&id=10qQjYYZBdcRAvOiM6Xa_vFZmGQOGuvH1",  # noqa: E501
    #index_col=0,
    na_values=[0,-999]
).dropna()
ldf = gpd.GeoDataFrame(light, geometry=geopandas.points_from_xy(light.lon, light.lat))



In [160]:

join1 = gpd.sjoin(gdf,ldf, how="inner", predicate='intersects')
column_mapping = {
    'lon_left': 'lon',
    'lat_left': 'lat',
    'City_left': 'City', # change
}

join1 = join1.rename(columns=column_mapping).drop(columns = ['index_right','lat_right', 'lon_right'])


join1

,ID,lon,lat,houseprice,City,geometry,X_0,X_1,X_2,X_3,...,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,X_255
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
982,"88,74",35.263528,0.520835,1.666015,Eldoret,POINT (35.26353 0.52083),inf,inf,inf,inf,...,inf,0.056259,inf,inf,inf,inf,inf,inf,inf,0.047500
994,"87,83",35.271710,0.521744,0.818014,Eldoret,POINT (35.27171 0.52174),3.812636e+33,inf,inf,inf,...,inf,0.085263,inf,inf,inf,inf,inf,inf,inf,0.067925
1005,"86,78",35.267164,0.522653,1.026086,Eldoret,POINT (35.26716 0.52265),inf,inf,inf,inf,...,inf,0.173145,inf,inf,inf,inf,inf,inf,inf,0.123826
1016,"85,73",35.262619,0.523562,3.819634,Eldoret,POINT (35.26262 0.52356),inf,inf,inf,inf,...,inf,0.215017,inf,inf,inf,inf,inf,inf,inf,0.155954
1027,"84,68",35.258073,0.524471,7.963232,Eldoret,POINT (35.25807 0.52447),3.297079e-04,inf,inf,inf,...,inf,0.027971,inf,inf,inf,inf,inf,inf,inf,0.022389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573,"27,66",37.067096,-1.025716,7.017600,Thika,POINT (37.06710 -1.02572),inf,0.055692,inf,0.157183,...,inf,inf,inf,inf,6.039991e+32,inf,inf,inf,inf,inf
1574,"27,78",37.078005,-1.025716,7.017600,Thika,POINT (37.07800 -1.02572),inf,inf,inf,0.057362,...,inf,inf,inf,inf,1.893345e-01,inf,inf,inf,inf,inf
1590,"26,72",37.072550,-1.024807,5.070685,Thika,POINT (37.07255 -1.02481),inf,0.019662,inf,0.054753,...,inf,inf,0.08745,inf,7.322688e+32,inf,inf,inf,inf,inf


In [161]:
NPARTITIONS = 250

ddf = dask_geopandas.from_geopandas(join1, npartitions=1)
#hd = ddf.hilbert_distance().compute()
#gdf["hd"] = hd
#gdf = gdf.sort_values("hd")

#dgdf = dask_geopandas.from_geopandas(gdf, npartitions=NPARTITIONS, sort=False)

In [164]:
%%time

with Client(n_workers=16) as client:
    print(client.dashboard_link)
    meta = ddf._meta.assign(stac_item=[])
    df2 = ddf.map_partitions(query_7, meta=meta).compute()

/user/zhaxinge@upenn.edu/proxy/8787/status
Returned 2 Items
CPU times: user 1.03 s, sys: 993 ms, total: 2.02 s
Wall time: 9.67 s


In [165]:
%%time

with Client(n_workers=16) as client:
    print(client.dashboard_link)
    meta = ddf._meta.assign(stac_item=[])
    df5 = ddf.map_partitions(query_8, meta=meta).compute()

/user/zhaxinge@upenn.edu/proxy/8787/status
Returned 2 Items
CPU times: user 564 ms, sys: 830 ms, total: 1.39 s
Wall time: 7.92 s


# Extract features

In [152]:

class CustomRGBDataset(Dataset):
    def __init__(self,points,fns_r,fns_g,fns_b):
        self.red_dataset = CustomDataset(points, fns_r)
        self.green_dataset = CustomDataset(points, fns_g)
        self.blue_dataset = CustomDataset(points, fns_b)
        self.points = points
        
        assert len(self.red_dataset)==len(self.green_dataset)==len(self.blue_dataset)
        
        self.length=len(self.red_dataset)
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        return torch.cat([self.red_dataset[idx], self.green_dataset[idx], self.blue_dataset[idx]],dim=0)
    
    

In [196]:
df6 = df5.dropna(subset=["stac_item"]).sort_values(["lat", "lon"], ascending=[False, True])
df4 = df2.dropna(subset=["stac_item"]).sort_values(["lat", "lon"], ascending=[False, True])
join = gpd.sjoin(df4,df6, how="inner", predicate='intersects')

column_mapping = {
    'lon_left': 'lon',
    'lat_left': 'lat',
    'City_left': 'City', # change
    'houseprice_left':'houseprice'
}

join = join.rename(columns=column_mapping).drop(columns = ['index_right','lat_right', 'lon_right'])
join

,ID_left,lon,lat,houseprice,City,geometry,X_0_left,X_1_left,X_2_left,X_3_left,...,X_247_right,X_248_right,X_249_right,X_250_right,X_251_right,X_252_right,X_253_right,X_254_right,X_255_right,stac_item_right
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1684,"26,44",35.236255,0.577196,1.612428,Eldoret,POINT (35.23626 0.57720),inf,inf,inf,inf,...,0.193522,inf,inf,inf,inf,inf,inf,inf,inf,"{'id': 'LC08_L2SP_170060_20130419_02_T1', 'bbo..."
1661,"28,40",35.232619,0.575378,3.444904,Eldoret,POINT (35.23262 0.57538),inf,inf,inf,inf,...,0.505238,inf,inf,inf,inf,inf,inf,inf,inf,"{'id': 'LC08_L2SP_170060_20130419_02_T1', 'bbo..."
1650,"29,45",35.237164,0.574469,3.709020,Eldoret,POINT (35.23716 0.57447),inf,inf,inf,inf,...,0.544518,inf,inf,inf,inf,inf,inf,inf,inf,"{'id': 'LC08_L2SP_170060_20130419_02_T1', 'bbo..."
1639,"30,50",35.241710,0.573560,0.260606,Eldoret,POINT (35.24171 0.57356),inf,inf,inf,inf,...,0.539423,inf,inf,inf,inf,inf,inf,inf,inf,"{'id': 'LC08_L2SP_170060_20130419_02_T1', 'bbo..."
1627,"31,41",35.233528,0.572651,4.250277,Eldoret,POINT (35.23353 0.57265),inf,inf,inf,inf,...,0.601870,inf,inf,inf,inf,inf,inf,inf,inf,"{'id': 'LC08_L2SP_170060_20130419_02_T1', 'bbo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,"115,93",37.241005,-1.480036,0.005530,Machakos,POINT (37.24101 -1.48004),inf,inf,inf,inf,...,inf,inf,inf,inf,inf,2.502789e+33,inf,inf,inf,"{'id': 'LC08_L2SP_168061_20131115_02_T1', 'bbo..."
1131,"116,87",37.235551,-1.480945,0.101554,Machakos,POINT (37.23555 -1.48094),inf,inf,inf,inf,...,inf,inf,inf,inf,inf,6.477690e-01,inf,inf,inf,"{'id': 'LC08_L2SP_168061_20131115_02_T1', 'bbo..."
1108,"119,96",37.243733,-1.483671,0.107196,Machakos,POINT (37.24373 -1.48367),inf,inf,inf,inf,...,inf,inf,inf,inf,inf,6.072096e-01,inf,inf,inf,"{'id': 'LC08_L2SP_168061_20131115_02_T1', 'bbo..."


In [197]:
from pystac import Item
from pystac.extensions.eo import EOExtension as eo
def find_asset_by_band_common_name(items, common_name):
    assets = []
    for item in items:
        for asset in item.assets.values():
            asset_bands = eo.ext(asset).bands
            if asset_bands and asset_bands[0].common_name == common_name:
                assets.append(asset.href)
    return assets
    raise KeyError(f"{common_name} band not found")
    


matching_items = []

for item in join.stac_item_left.tolist():
    signed_item = pc.sign(Item.from_dict(item))
    matching_items.append(signed_item)

asset_r1 = [pc.sign(asset_href) for asset_href in find_asset_by_band_common_name(matching_items, "red")]
asset_g1 = [pc.sign(asset_href) for asset_href in find_asset_by_band_common_name(matching_items, "green")]
asset_b1 = [pc.sign(asset_href) for asset_href in find_asset_by_band_common_name(matching_items, "blue")]

matching_nir1 = [
    pc.sign(asset_href) for asset_href in find_asset_by_band_common_name(matching_items, "nir08")
]

In [198]:
from pystac import Item
from pystac.extensions.eo import EOExtension as eo
def find_asset_by_band_common_name(items, common_name):
    assets = []
    for item in items:
        for asset in item.assets.values():
            asset_bands = eo.ext(asset).bands
            if asset_bands and asset_bands[0].common_name == common_name:
                assets.append(asset.href)
    return assets
    raise KeyError(f"{common_name} band not found")
    


matching_items = []

for item in join.stac_item_right.tolist():
    signed_item = pc.sign(Item.from_dict(item))
    matching_items.append(signed_item)

asset_r2 = [pc.sign(asset_href) for asset_href in find_asset_by_band_common_name(matching_items, "red")]
asset_g2 = [pc.sign(asset_href) for asset_href in find_asset_by_band_common_name(matching_items, "green")]
asset_b2 = [pc.sign(asset_href) for asset_href in find_asset_by_band_common_name(matching_items, "blue")]

matching_nir2 = [
    pc.sign(asset_href) for asset_href in find_asset_by_band_common_name(matching_items, "nir08")
]

In [209]:

columns_to_exclude = ['ID', 'lon', 'lat', 'houseprice', 'City', 'geometry', 'stac_item']
selected_columns = [col for col in df6.columns if col not in columns_to_exclude]
df7 = df6[selected_columns]
df7

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,X_255
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1684,inf,inf,inf,inf,0.210824,inf,inf,inf,inf,inf,...,inf,0.193522,inf,inf,inf,inf,inf,inf,inf,inf
1661,inf,inf,inf,inf,0.575936,inf,inf,inf,inf,inf,...,inf,0.505238,inf,inf,inf,inf,inf,inf,inf,inf
1650,inf,inf,inf,inf,0.578821,inf,inf,inf,inf,inf,...,inf,0.544518,inf,inf,inf,inf,inf,inf,inf,inf
1639,inf,inf,inf,inf,0.568637,inf,inf,0.267347,inf,inf,...,inf,0.539423,inf,inf,inf,inf,inf,inf,inf,inf
1627,inf,inf,inf,inf,0.578514,inf,inf,inf,inf,inf,...,inf,0.601870,inf,inf,inf,inf,inf,inf,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,inf,inf,inf,inf,inf,inf,inf,2.146536,inf,inf,...,inf,inf,inf,inf,inf,inf,2.502789e+33,inf,inf,inf
1131,inf,inf,inf,inf,inf,inf,inf,0.919349,1.236867e+33,inf,...,inf,inf,inf,inf,inf,inf,6.477690e-01,inf,inf,inf
1108,inf,inf,inf,inf,inf,inf,inf,0.345705,1.709304e-01,inf,...,inf,inf,inf,inf,inf,inf,6.072096e-01,inf,inf,inf


In [211]:
points = join[["lon", "lat"]].to_numpy()
houseprice_log = np.log10(join["houseprice"].to_numpy() + 1)

# Create a group array based on the "city" column
groups = join["City"].to_numpy()

# Perform leave-one-city-out splitting
logo = LeaveOneGroupOut()
train_sets = []
test_sets = []
group_scores = {}



# Define the hyperparameter search space
param_distributions = {
    'alpha': np.logspace(-8, 8, base=10, num=17),
    'solver': ['auto']
}

for train_indices, test_indices in logo.split(points, groups=groups):
    train_sets.append(train_indices)
    test_sets.append(test_indices)
    city = groups[test_indices[0]]
    print(city)
    train_dataset2 = CustomRGBDataset(points[train_indices], [asset_r1[idx] for idx in train_indices], [asset_b1[idx] for idx in train_indices], [asset_g1[idx] for idx in train_indices])
    test_dataset2 = CustomRGBDataset(points[test_indices], [asset_r1[idx] for idx in test_indices], [asset_b1[idx] for idx in test_indices], [asset_g1[idx] for idx in test_indices])
    train_dataset1 = CustomDataset(points[train_indices], [matching_nir1[idx] for idx in train_indices])
    test_dataset1 = CustomDataset(points[test_indices], [matching_nir1[idx] for idx in train_indices])
    
    train_dataset4 = CustomRGBDataset(points[train_indices], [asset_r2[idx] for idx in train_indices], [asset_b2[idx] for idx in train_indices], [asset_g2[idx] for idx in train_indices])
    test_dataset4 = CustomRGBDataset(points[test_indices], [asset_r2[idx] for idx in test_indices], [asset_b2[idx] for idx in test_indices], [asset_g2[idx] for idx in test_indices])
    train_dataset3 = CustomDataset(points[train_indices], [matching_nir2[idx] for idx in train_indices])
    test_dataset3 = CustomDataset(points[test_indices], [matching_nir2[idx] for idx in train_indices])
    
    len(train_dataset4)
    model1 = RCF(num_features=128, num_input_channels=1).eval().to(device) 
    model2 = RCF(num_features=256, num_input_channels=3).eval().to(device)
    # Extract features from CustomDataset for train and test datasets
    x_train1 = extract_features(train_dataset1,model1,num_features=128)
    x_train2 = extract_features(train_dataset2,model2,num_features=256)
    x_train4 = extract_features(train_dataset3,model1,num_features=128)
    x_train5 = extract_features(train_dataset4,model2,num_features=256)
    x_train3 = df7.iloc[train_indices]
    
    x_test1 = extract_features(test_dataset1, model1,num_features=128)
    x_test2 = extract_features(test_dataset2, model2,num_features=256)
    x_test3 = df7.iloc[test_indices]
    x_test4 = extract_features(test_dataset3, model1,num_features=128)
    x_test5 = extract_features(test_dataset4, model2,num_features=256)
    
        # Add nightlight_log to the training set
    x_train = np.concatenate((x_train1,x_train2,x_train3,x_train4,x_train5), axis=1)
    print(x_train.shape)
    x_train[np.isinf(x_train)] = 0
    x_test = np.concatenate((x_test1,x_test2,x_test3,x_test4,x_test5), axis=1)
    x_test[np.isinf(x_test)] = 0

    y_train = houseprice_log.copy()[train_indices]
    y_test = houseprice_log.copy()[test_indices]
    print(x_train2.shape)
    print(y_train.shape)

    # Perform random search for hyperparameter tuning
    ridge = Ridge()
    ridge_random = RandomizedSearchCV(ridge, param_distributions, cv=5, n_iter=20, random_state=42)
    ridge_random.fit(x_train, y_train)
    
    # Get the best hyperparameters
    best_alpha = ridge_random.best_params_['alpha']
    best_solver = ridge_random.best_params_['solver']
    
    # Initialize Ridge Regression with the best hyperparameters
    ridge_cv_best = Ridge(alpha=best_alpha, solver=best_solver)
    ridge_cv_best.fit(x_train, y_train)
    
    # Evaluate the model on the test dataset
    test_predictions = ridge_cv_best.predict(x_test)
    test_mse = mean_squared_error(y_test, test_predictions)
    test_score = r2_score(y_test, test_predictions)
    
    # Calculate the train score on the current city group
    train_predictions = ridge_cv_best.predict(x_train)
    train_mse = mean_squared_error(y_train, train_predictions)
    train_score = r2_score(y_train, train_predictions)
    
    # Store the test score and train score for the current city group
    group_scores[city] = {'train_score': train_score, 'train_mse':train_mse,'test_score': test_score, 'test_mse':test_mse}

Eldoret


/srv/conda/envs/notebook/lib/python3.9/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


0/487 -- 0.00% -- 0.54 seconds
0/487 -- 0.00% -- 0.80 seconds
0/487 -- 0.00% -- 0.49 seconds
0/487 -- 0.00% -- 0.68 seconds
0/139 -- 0.00% -- 0.44 seconds
0/139 -- 0.00% -- 0.66 seconds
0/139 -- 0.00% -- 0.53 seconds
0/139 -- 0.00% -- 0.74 seconds
(487, 1024)
(487, 256)
(487,)


/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 17 is smaller than n_iter=20. Running 17 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-sq

Embu


...

/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 17 is smaller than n_iter=20. Running 17 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-sq

Thika


/srv/conda/envs/notebook/lib/python3.9/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


0/483 -- 0.00% -- 0.49 seconds
0/483 -- 0.00% -- 0.69 seconds
0/483 -- 0.00% -- 0.50 seconds
0/483 -- 0.00% -- 0.76 seconds
0/143 -- 0.00% -- 0.35 seconds
0/143 -- 0.00% -- 0.68 seconds
0/143 -- 0.00% -- 0.35 seconds
0/143 -- 0.00% -- 0.75 seconds
(483, 1024)
(483, 256)
(483,)


/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 17 is smaller than n_iter=20. Running 17 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:251: UserWarning: Singular matrix in solving dual problem. Using least-sq

In [212]:
# Print the evaluation scores for each city group
for city, score in group_scores.items():
    print(f"City: {city}, Test Score: {score}")

City: Eldoret, Test Score: {'train_score': 0.45026135136457734, 'train_mse': 0.12665675017680916, 'test_score': -193447.33441305574, 'test_mse': 17353.683588948377}
City: Embu, Test Score: {'train_score': 0.5054895229835313, 'train_mse': 0.09925765271510177, 'test_score': -98501946.93724738, 'test_mse': 3849442.586700669}
City: Kakamega, Test Score: {'train_score': 0.5172548390283467, 'train_mse': 0.09250245573109424, 'test_score': -571.565024272493, 'test_mse': 142.90143101237368}
City: Kericho, Test Score: {'train_score': 0.4606000418863061, 'train_mse': 0.10083121181685512, 'test_score': -2184.442910793591, 'test_mse': 719.7233649723589}
City: Kisumu, Test Score: {'train_score': 0.4617379746749678, 'train_mse': 0.10815346065033236, 'test_score': -484.9276004572664, 'test_mse': 5.012523595872463}
City: Kitui, Test Score: {'train_score': 0.46712193646741007, 'train_mse': 0.1075100977337499, 'test_score': -694.718287354032, 'test_mse': 53.580222341333716}
City: Machakos, Test Score: {'

In [213]:
pd.DataFrame.from_dict(group_scores,  orient='index')

,train_score,train_mse,test_score,test_mse
Eldoret,0.450261,0.126657,-1.934473e+05,1.735368e+04
Embu,0.505490,0.099258,-9.850195e+07,3.849443e+06
Kakamega,0.517255,0.092502,-5.715650e+02,1.429014e+02
Kericho,0.460600,0.100831,-2.184443e+03,7.197234e+02
Kisumu,0.461738,0.108153,-4.849276e+02,5.012524e+00
Kitui,0.467122,0.107510,-6.947183e+02,5.358022e+01
Machakos,0.365781,0.127538,-4.137600e+06,2.791110e+05
Nairobi,0.461406,0.107861,-9.424587e+03,8.264431e+02
Nakuru,0.418857,0.104660,-4.678076e+06,6.814108e+05
Nyeri,0.474576,0.105764,-3.538281e+03,1.165648e+02


# Full datasset

In [ ]:
dataset = CustomDataset(points, matching_urls)

dataloader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=False,
    num_workers=os.cpu_count() * 2,
    collate_fn=lambda x: x,
    pin_memory=False,
)

In [ ]:
x_all = np.zeros((points.shape[0], num_features), dtype=float)

tic = time.time()
i = 0
for images in dataloader:
    for image in images:

        if image is not None:
            # A full image should be ~101x101 pixels (i.e. ~1km^2 at a 10m/px spatial
            # resolution), however we can receive smaller images if an input point
            # happens to be at the edge of a S2 scene (a literal edge case). To deal
            # with these (edge) cases we crudely drop all images where the spatial
            # dimensions aren't both greater than 20 pixels.
            if image.shape[1] >= 20 and image.shape[2] >= 20:
                image = image.to(device)
                with torch.no_grad():
                    feats = model(image.unsqueeze(0)).cpu().numpy()

                x_all[i] = feats
            else:
                # this happens if the point is close to the edge of a scene
                # (one or both of the spatial dimensions of the image are very small)
                pass
        else:
            pass  # this happens if we do not find a S2 scene for some point

        if i % 1000 == 0:
            print(
                f"{i}/{points.shape[0]} -- {i / points.shape[0] * 100:0.2f}%"
                + f" -- {time.time()-tic:0.2f} seconds"
            )
            tic = time.time()
        i += 1

In [ ]:
y_all = houseprice_log.copy()

In [ ]:
nofeature_mask = ~(x_all.sum(axis=1) == 0)

In [ ]:
x_all.shape, y_all.shape

In [ ]:
x_all = x_all[nofeature_mask]
y_all = y_all[nofeature_mask]

In [ ]:
x_all.shape, y_all.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x_all, y_all, test_size=0.2, random_state=0
)

In [ ]:
ridge_cv_random = RidgeCV(cv=5, alphas=np.logspace(-8, 8, base=10, num=17))
ridge_cv_random.fit(x_train, y_train)

In [ ]:
print(f"Validation R2 performance {ridge_cv_random.best_score_:0.2f}")

In [ ]:
y_pred = np.maximum(ridge_cv_random.predict(x_test), 0)

plt.figure()
plt.scatter(y_pred, y_test, alpha=0.2, s=4)
plt.xlabel("Predicted", fontsize=15)
plt.ylabel("Ground Truth", fontsize=15)
plt.title(r"$\log_{10}(1 + $houseprice$)$", fontsize=15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlim([0, 6])
plt.ylim([0, 6])

plt.text(
    0.5,
    5,
    s="R$^2$ = %0.2f" % (r2_score(y_test, y_pred)),
    fontsize=15,
    fontweight="bold",
)
m, b = np.polyfit(y_pred, y_test, 1)
plt.plot(y_pred, m * y_pred + b, color="black")
plt.gca().spines.right.set_visible(False)
plt.gca().spines.top.set_visible(False)

plt.show()
plt.close()

In [ ]:
spearmanr(y_pred, y_test)

In [ ]:
import statsmodels.api as sm

# Perform OLS regression on the whole dataset
x_train = sm.add_constant(x_train)  # Add constant term for the intercept
ols_model = sm.OLS(y_train, x_train)
ols_results = ols_model.fit()

# Get the summary tables
summary_tables = ols_results.summary().tables

# Print the upper part of the OLS Regression Results
print(summary_tables[0])


In [ ]:

# Perform OLS regression on the whole dataset
x_test = sm.add_constant(x_test)  # Add constant term for the intercept
ols_model = sm.OLS(y_test, x_test)
ols_results = ols_model.fit()

# Get the summary tables
summary_tables = ols_results.summary().tables

# Print the upper part of the OLS Regression Results
print(summary_tables[0])

In [ ]:
points = points[nofeature_mask]

In [ ]:
split_lon = np.percentile(points[:, 0], 80)
train_idxs = np.where(points[:, 0] <= split_lon)[0]
test_idxs = np.where(points[:, 0] > split_lon)[0]

x_train = x_all[train_idxs]
x_test = x_all[test_idxs]
y_train = y_all[train_idxs]
y_test = y_all[test_idxs]

In [ ]:
plt.figure()
plt.scatter(points[:, 0], points[:, 1], c=y_all, s=1)
plt.vlines(
    split_lon,
    ymin=points[:, 1].min(),
    ymax=points[:, 1].max(),
    color="black",
    linewidth=4,
)
plt.axis("off")
plt.show()
plt.close()

In [ ]:
ridge_cv = RidgeCV(cv=5, alphas=np.logspace(-8, 8, base=10, num=17))
ridge_cv.fit(x_train, y_train)

In [ ]:
print(f"Validation R2 performance {ridge_cv.best_score_:0.2f}")

In [ ]:
y_pred = np.maximum(ridge_cv.predict(x_test), 0)

plt.figure()
plt.scatter(y_pred, y_test, alpha=0.2, s=4)
plt.xlabel("Predicted", fontsize=15)
plt.ylabel("Ground Truth", fontsize=15)
plt.title(r"$\log_{10}(1 + $people$/$km$^2)$", fontsize=15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlim([0, 6])
plt.ylim([0, 6])

plt.text(
    0.5,
    5,
    s="R$^2$ = %0.2f" % (r2_score(y_test, y_pred)),
    fontsize=15,
    fontweight="bold",
)
m, b = np.polyfit(y_pred, y_test, 1)
plt.plot(y_pred, m * y_pred + b, color="black")
plt.gca().spines.right.set_visible(False)
plt.gca().spines.top.set_visible(False)

plt.show()
plt.close()

In [ ]:
spearmanr(y_test, y_pred)

In [ ]:
bins = np.linspace(0, 5, num=50)

plt.figure()
plt.hist(y_train, bins=bins)
plt.ylabel("Frequency")
plt.xlabel(r"$\log_{10}(1 + $people$/$km$^2)$")
plt.title("Train points -- western US")
plt.gca().spines.right.set_visible(False)
plt.gca().spines.top.set_visible(False)
plt.show()
plt.close()

plt.figure()
plt.hist(y_test, bins=bins)
plt.ylabel("Frequency")
plt.xlabel(r"$\log_{10}(1 + $people$/$km$^2)$")
plt.title("Test points -- eastern US")
plt.gca().spines.right.set_visible(False)
plt.gca().spines.top.set_visible(False)
plt.show()
plt.close()

In [ ]:
y_pred = np.maximum(ridge_cv.predict(x_test), 0)

plt.figure()
plt.scatter(y_pred, y_test, alpha=0.2, s=4)
plt.xlabel("Predicted", fontsize=15)
plt.ylabel("Ground Truth", fontsize=15)
plt.title(r"$\log_{10}(1 + $people$/$km$^2)$", fontsize=15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlim([0, 6])
plt.ylim([0, 6])

plt.text(
    0.5,
    5,
    s="R$^2$ = %0.2f" % (r2_score(y_test, y_pred)),
    fontsize=15,
    fontweight="bold",
)
m, b = np.polyfit(y_pred, y_test, 1)
plt.plot(y_pred, m * y_pred + b, color="black")
plt.gca().spines.right.set_visible(False)
plt.gca().spines.top.set_visible(False)

plt.show()
plt.close()

# LIGHT

In [ ]:
import geopandas as gpd
join = gpd.sjoin(gdf,ldf, how="inner", predicate='intersects')
desired_columns = ['ID_left', 'lon_left', 'lat_left', 'houseprice', 'City_left', 'geometry','nightlight']
result_df = join[desired_columns]
# Rename the columns in the resulting DataFrame
column_mapping = {
    'ID_left': 'ID',
    'lon_left': 'lon',
    'lat_left': 'lat',
    'City_left': 'City'
}

result_df = result_df.rename(columns=column_mapping)


result_df

In [ ]:
np.unique(result_df['City'])

In [ ]:
nightlight_log = np.log10(result_df['nightlight'] + 1)
nightlight_log .plot.hist()